<a href="https://colab.research.google.com/github/ynusinovich/streamflow-prediction/blob/main/ARE/HYSETS_DARTS_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
from google.colab import drive
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/Streamflow Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
import xarray as xr
import pandas as pd
import numpy as np

In [108]:
!pip install pyyaml==5.4.1
!pip install darts
!pip install -U matplotlib==3.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 7.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.3 MB/s 
     |████████████████████████████████| 569 kB 46.4 MB/s 
     |████████████████████████████████| 1.8 MB 46.8 MB/s 
     |████████████████████████████████| 8.9 MB 10.5 MB/s 
     |████████████████████████████████| 708 kB 69.0 MB/s 
     |████████████████████████████████| 77 kB 5.2 MB/s 
     |████████████████████████████████| 9.8 MB 22.8 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 76.8 MB 1.2 MB/s 
     |████████████████████████████████| 11.2 MB 24.4 MB/s 
     |████████████████████████████████| 1.6 MB 51.9 MB/s 
     |████████████████████████████████| 960 kB 41.2 MB/s 
     |████████████████████████████████| 5.9 MB 40.3 MB/s 
     |████████████████████████████████| 529 kB 67.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 7.4 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.3
    Uninstalling matplotlib-3.5.3:
      Successfully uninstalled matplotlib-3.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nfoursid 1.0.1 requires matplotlib>=3.3, but you have matplotlib 3.1.3 which is incompatible.
darts 0.22.0 requires matplotlib>=3.3.0, but you have matplotlib 3.1.3 which is incompatible.


In [8]:
#@title Load Watershed Properties and Data

In [163]:
# load static properties
lcrop = True

# load properties
if lcrop:
    wsp = pd.read_csv(data_root+'/HYSETS_watershed_properties_cleaned_prairies.txt')
else:
    wsp = pd.read_csv(data_root+'/HYSETS_watershed_properties_cleaned.txt')
# some cleaning
wsp.drop(columns='Unnamed: 0', inplace=True)
wsp['watershed'] = wsp['Watershed_ID']
wsp.set_index('watershed', drop=True, inplace=True)
wsp

,Watershed_ID,Source,Name,Official_ID,Centroid_Lat_deg_N,Centroid_Lon_deg_E,Drainage_Area_km2,Elevation_m,Slope_deg,Gravelius,...,Land_Use_Crops_frac,Land_Use_Snow_Ice_frac,Flag_Land_Use_Extraction,Permeability_logk_m2,Porosity_frac,Flag_Subsoil_Extraction,Hydrometric_station_latitude,Hydrometric_station_longitude,Flag_Natural_Boundaries,Flag_ALL
watershed,,,,,,,,,,,,,,,,,,,,,
865,865,HYDAT,WILLOW CREEK AT HIGHWAY NO. 811,05AB046,49.75476,-113.40730,2509.8584,1262.0,5.7682,1.6076,...,0.2563,0.0000,1,-15.262326,0.186644,1,49.75476,-113.40730,1,1
867,867,HYDAT,LITTLE BOW RIVER ABOVE TRAVERS RESERVOIR,05AC034,50.20417,-112.97780,3490.0697,1034.6,1.2383,1.8040,...,0.8288,0.0000,1,-15.200000,0.190000,1,50.20417,-112.97780,1,1
868,868,HYDAT,LITTLE BOW RIVER AT HIGHWAY NO. 533,05AC930,50.35417,-113.54310,791.6734,1026.4,0.6497,1.4250,...,0.9049,0.0000,1,-15.200000,0.190000,1,50.35417,-113.54310,1,1
869,869,HYDAT,WATERTON RIVER NEAR WATERTON PARK,05AD003,49.11366,-113.83940,612.6766,1838.1,22.5613,1.6875,...,0.0011,0.0057,1,-15.231325,0.106944,1,49.11366,-113.83940,1,1
877,877,HYDAT,BELLY RIVER NEAR MOON RIVER ROAD,05AD934,49.72870,-113.12470,3584.4565,1382.8,8.0552,1.7799,...,0.4126,0.0022,1,-15.359595,0.160534,1,49.72870,-113.12470,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1230,HYDAT,NETLEY CREEK NEAR PETERSFIELD,05OJ008,50.32722,-97.04417,641.0558,256.3,0.2838,1.2551,...,0.5614,0.0000,1,-11.800000,0.060000,1,50.32722,-97.04417,1,1
1273,1273,HYDAT,ICELANDIC RIVER NEAR RIVERTON,05SC002,50.96472,-97.03747,1236.7307,252.9,0.2952,1.6384,...,0.5162,0.0000,1,-11.803577,0.060137,1,50.96472,-97.03747,1,1
1274,1274,HYDAT,FISHER RIVER NEAR DALLAS,05SD003,51.35617,-97.51222,1706.3842,258.5,0.4215,1.5097,...,0.2668,0.0000,1,-11.800000,0.060000,1,51.35617,-97.51222,1,1


In [164]:
# load NetCDF file lazily
if lcrop:
    xds = xr.open_dataset(data_root+'/HYSETS_2020_QC_stations_mod2_crop.nc', chunks={'watershed':1})
else:
    xds = xr.open_dataset(data_root+'/HYSETS_2020_QC_stations_mod2.nc', chunks={'watershed':1})
# use watershed IDs to slice out watersheds of interest
xds = xds.sel(watershed=wsp['Watershed_ID'].values)
# load properties into memory
for var in xds.data_vars.values():
    if var.ndim == 1:
        var.load()
xds

<xarray.Dataset>
Dimensions:                     (time: 25202, watershed: 112)
Coordinates:
  * time                        (time) datetime64[ns] 1950-01-01 ... 2018-12-31
  * watershed                   (watershed) int32 865 867 868 ... 1274 1283 2365
Data variables: (12/15)
    watershedID                 (watershed) float64 865.0 867.0 ... 2.365e+03
    drainage_area               (watershed) float64 2.51e+03 ... 1.97e+03
    drainage_area_GSIM          (watershed) float64 nan nan ... nan 1.995e+03
    flag_GSIM_boundaries        (watershed) float64 0.0 0.0 0.0 ... 0.0 0.0 1.0
    flag_artificial_boundaries  (watershed) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    centroid_lat                (watershed) float64 49.75 50.2 ... 53.82 49.02
    ...                          ...
    discharge                   (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    pr                          (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmax                      (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmin                      (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    data_mask                   (watershed, time) bool dask.array<chunksize=(1, 25202), meta=np.ndarray>
    mask_300                    (watershed, time) uint16 dask.array<chunksize=(1, 25202), meta=np.ndarray>
Attributes: (12/13)
    title:          Hydrometeorological Sandbox of the École de technologie s...
    summary:        Hydrometeorological Sandbox of the École de technologie s...
    institution:    Hydrology, Climate and Climate Change Laboratory (HC3) at...
    institute_id:   HC3-ETS
    contact:        Richard Arsenault: richard.arsenault@etsmtl.ca
    date_created:   2020-05-26
    ...             ...
    featureType:    timeSeries
    cdm_data_type:  station
    license:        ODC-BY
    keywords:       HYSETS, GHCND, GSIM, hydrology, North America, streamflow...
    conventions:    CF-1.6, ACDD-1.3
    activity:       HYSETS

In [165]:
#@title Extract Periods of Consecutive Valid Data

In [211]:
data_mask = xds.data_mask
mask_300 = xds.mask_300

# some reorganization of data
ts_varnames = ('discharge', )
cov_varnames = ('pr', 'tasmin', 'tasmax')
ts_xds = xr.Dataset({varname:xds[varname] for varname in ts_varnames+cov_varnames})
ts_xds

<xarray.Dataset>
Dimensions:    (time: 25202, watershed: 112)
Coordinates:
  * time       (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2018-12-31
  * watershed  (watershed) int32 865 867 868 869 877 ... 1273 1274 1283 2365
Data variables:
    discharge  (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    pr         (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmin     (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmax     (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>

In [167]:
# determine means and variances by watershed for static covariats and normalization
sds = xr.Dataset(data_vars=None, coords={'watershed':xds.watershed})
for varname in ['discharge', 'pr', 'tasmin', 'tasmax']:
    data_var = xds[varname]
    # compute means/std
    sds[varname+'_mean'] = data_var.mean(dim=('time',)).load()
    sds[varname+'_std']  = data_var.std(dim=('time',)).load()

# add to properties dataframe
stats_df = sds.to_dataframe()
stats_df

,discharge_mean,discharge_std,pr_mean,pr_std,tasmin_mean,tasmin_std,tasmax_mean,tasmax_std
watershed,,,,,,,,
865,6.212508,16.222912,1.397961,3.986137,-2.299139,10.016271,10.625340,11.139233
867,2.730594,3.131016,1.212713,3.467988,-2.114604,10.326642,11.219891,11.972828
868,1.965355,2.361048,1.236688,3.745683,-2.588431,10.382421,10.750421,12.201200
869,17.894640,26.214912,2.954037,6.186049,-1.649298,9.022515,9.862430,10.689588
877,27.447258,46.197613,1.963749,4.449354,-1.377792,9.549591,10.827139,11.038117
...,...,...,...,...,...,...,...,...
1230,1.662630,5.325261,1.434288,4.343831,-3.432139,14.134577,7.724002,14.932825
1273,3.228191,10.444004,1.395137,4.093546,-4.080882,14.225872,7.342315,15.035342
1274,2.721829,8.511021,1.419544,4.080548,-4.674377,14.303836,7.375019,15.017003


In [168]:
## assemble static dataframes with means & std
# join with existing properties by index
wsp_ms = wsp.join(stats_df, on=None, how='left')
# validate
assert (np.isnan(wsp_ms['discharge_mean'].values)).any() == False

# dropable columns
str_cols = ['Source', 'Name', 'Official_ID']
flag_cols = ['Flag_ALL', 'Flag_Shape_Extraction', 'Flag_Terrain_Extraction', 'Flag_Land_Use_Extraction', 'Flag_Subsoil_Extraction', 'Flag_Natural_Boundaries']
drop_cols = ['Watershed_ID', 'Centroid_Lon_deg_E', 'Hydrometric_station_latitude', 'Hydrometric_station_longitude'] + flag_cols + str_cols

# trim props for static covariates
print('\nDropping columns for traing data:\n', *drop_cols)
wsp_train = wsp_ms.drop(columns=drop_cols, inplace=False)
wsp_train


Dropping columns for traing data:
 Watershed_ID Centroid_Lon_deg_E Hydrometric_station_latitude Hydrometric_station_longitude Flag_ALL Flag_Shape_Extraction Flag_Terrain_Extraction Flag_Land_Use_Extraction Flag_Subsoil_Extraction Flag_Natural_Boundaries Source Name Official_ID


,Centroid_Lat_deg_N,Drainage_Area_km2,Elevation_m,Slope_deg,Gravelius,Perimeter,Aspect_deg,Land_Use_Forest_frac,Land_Use_Grass_frac,Land_Use_Wetland_frac,...,Permeability_logk_m2,Porosity_frac,discharge_mean,discharge_std,pr_mean,pr_std,tasmin_mean,tasmin_std,tasmax_mean,tasmax_std
watershed,,,,,,,,,,,,,,,,,,,,,
865,49.75476,2509.8584,1262.0,5.7682,1.6076,283.798,118.6022,0.2480,0.3854,0.0077,...,-15.262326,0.186644,6.212508,16.222912,1.397961,3.986137,-2.299139,10.016271,10.625340,11.139233
867,50.20417,3490.0697,1034.6,1.2383,1.8040,375.464,105.8134,0.0079,0.0923,0.0029,...,-15.200000,0.190000,2.730594,3.131016,1.212713,3.467988,-2.114604,10.326642,11.219891,11.972828
868,50.35417,791.6734,1026.4,0.6497,1.4250,141.224,163.4672,0.0009,0.0117,0.0003,...,-15.200000,0.190000,1.965355,2.361048,1.236688,3.745683,-2.588431,10.382421,10.750421,12.201200
869,49.11366,612.6766,1838.1,22.5613,1.6875,147.343,287.0749,0.5341,0.2608,0.0026,...,-15.231325,0.106944,17.894640,26.214912,2.954037,6.186049,-1.649298,9.022515,9.862430,10.689588
877,49.72870,3584.4565,1382.8,8.0552,1.7799,375.794,61.3698,0.2013,0.2598,0.0158,...,-15.359595,0.160534,27.447258,46.197613,1.963749,4.449354,-1.377792,9.549591,10.827139,11.038117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,50.32722,641.0558,256.3,0.2838,1.2551,111.934,112.7949,0.1996,0.1074,0.0525,...,-11.800000,0.060000,1.662630,5.325261,1.434288,4.343831,-3.432139,14.134577,7.724002,14.932825
1273,50.96472,1236.7307,252.9,0.2952,1.6384,202.881,78.8539,0.2345,0.1281,0.0277,...,-11.803577,0.060137,3.228191,10.444004,1.395137,4.093546,-4.080882,14.225872,7.342315,15.035342
1274,51.35617,1706.3842,258.5,0.4215,1.5097,219.568,49.3099,0.4043,0.1775,0.0465,...,-11.800000,0.060000,2.721829,8.511021,1.419544,4.080548,-4.674377,14.303836,7.375019,15.017003


In [236]:
%%time
# loop over stations
from darts import TimeSeries

lprint = True

ts_series_list = []
cov_series_list = []

# for wid in [1151]: 
# for wid in [915]: 
for wid in xds.watershed.values:

    ## identify indices pf valid periods
    ws_days = xds.mask_300.sel(watershed=wid).values
    wsd_del = np.diff(np.where(ws_days > 0, np.int8(1), np.int8(0),),)  # potentially larger arrays
    # find start points of valid periods
    start_idxs = np.nonzero(wsd_del > 0)[0] + 1
    if ws_days[0] > 0: start_idxs =np.concatenate(([0],start_idxs))
    # find end points of valid periods
    end_idxs = np.nonzero(wsd_del < 0)[0] + 1
    if ws_days[-1] > 0: end_idxs = np.concatenate((end_idxs,[ws_days.size]))
    assert end_idxs.size == start_idxs.size
    # verify length of each period
    prd_len = end_idxs - start_idxs
    #print(wid, prd_len)
    if lprint and prd_len.size > 0: 
        print("\nWatershed ID:", wid)
        #print(wid, start_idxs, end_idxs)
    assert ( prd_len >= 300 ).all()
    # test even more stringently
    test_idxs = np.stack((start_idxs, end_idxs), axis=1)
    assert ( np.diff( test_idxs.flatten() ) > 0 ).all(), test_idxs

    ## Generate Timeseries objects
    # select by watershed
    ts_ws = ts_xds.sel(watershed=wid)
    stat_ws = wsp_train.loc[wid]  # indexed by watershed ID
    
    # loop over time periods and generate TimeSeries
    for si,ei in zip(start_idxs, end_idxs):
        if lprint: print('Period:',si,ei)

        # slice time
        tsx = ts_ws.isel(time=slice(si,ei))
        dti = pd.DatetimeIndex(tsx.time)
        # extract arrays and normalize
        ts_vars = dict()
        for varname,ts_var in tsx.data_vars.items():
            ts_vars[varname] = ( ts_var.values - stat_ws[varname+'_mean'] ) / stat_ws[varname+'_std']
        ts_vars['dayofyear'] = (dti.day_of_year.values - 183.) / 105.366  # sort of normalize

        # create TimeSeries
        ts_data = np.stack([ts_vars[varname] for varname in ts_varnames], axis=1)
        ts_series = TimeSeries.from_times_and_values(dti, ts_data, static_covariates=stat_ws)
        ts_series_list.append(ts_series)
        cov_data = np.stack([ts_vars[varname] for varname in cov_varnames+('dayofyear',)], axis=1)
        cov_series = TimeSeries.from_times_and_values(dti, cov_data, static_covariates=stat_ws)
        cov_series_list.append(cov_series)

print(f"\nGenerated {len(ts_series_list)} TimeSeries objects with Covariates.\n")


Watershed ID: 869
Period: 0 24106

Watershed ID: 911
Period: 5752 6909
Period: 20498 24106

Watershed ID: 915
Period: 2800 5752

Watershed ID: 933
Period: 6999 24472
Period: 24837 25202

Watershed ID: 992
Period: 20120 21550

Watershed ID: 993
Period: 7724 24837

Watershed ID: 1022
Period: 7726 23741

Watershed ID: 1023
Period: 22339 23741

Watershed ID: 1053
Period: 10286 23723
Period: 23724 24837

Watershed ID: 1089
Period: 15998 24472

Watershed ID: 1110
Period: 195 3926

Watershed ID: 1113
Period: 3926 17471

Watershed ID: 1114
Period: 8460 17897

Watershed ID: 1151
Period: 2251 20279
Period: 20506 23334
Period: 23391 24472

Watershed ID: 1153
Period: 4716 9862

Watershed ID: 1160
Period: 8035 24837

Watershed ID: 1161
Period: 8035 16222

Watershed ID: 1182
Period: 9921 17471

Watershed ID: 1188
Period: 8460 17167

Watershed ID: 1193
Period: 7729 17168

Watershed ID: 1198
Period: 4077 24837

Watershed ID: 1229
Period: 3195 6117

Watershed ID: 1274
Period: 4100 17167

Watershed ID: